In [26]:
import pandas as pd
import numpy as np
import math
df_val_pos = pd.read_csv('val_positive.txt', sep = '\s', header=None)
df_val_neg = pd.read_csv('val_negative.txt', sep = '\s', header=None)
df_test = pd.read_csv('test.txt', sep = '\s', header=None)
val_pos = np.array([df_val_pos[0], df_val_pos[1]])
val_neg = np.array([df_val_neg[0], df_val_neg[1]])
test = np.array([df_test[0], df_test[1]])

C:\Users\simon\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.
C:\Users\simon\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """
C:\Users\simon\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [3]:
import networkx as nx
G = nx.read_edgelist('training.txt')
len(G.nodes()) #5240

5240

In [4]:
nx.betweenness_centrality(G)

{'8193': 0.0,
 '16056': 0.003902934323090774,
 '24578': 0.00042857443102074436,
 '21968': 0.004943199494258445,
 '18297': 0.0018519171500315393,
 '16770': 0.0,
 '17038': 0.01789569028124117,
 '8195': 0.0006225931732552159,
 '2072': 0.006197413862386202,
 '20568': 0.0,
 '15258': 2.0536256782735806e-05,
 '19739': 0.0,
 '16389': 0.004547512420291664,
 '21379': 0.006863173365945859,
 '11015': 0.0024284915164918047,
 '23051': 0.001516802921086972,
 '20116': 0.001638220164948425,
 '15126': 7.856342824178312e-06,
 '21281': 0.010638809071191215,
 '17828': 3.0757544984381256e-05,
 '1265': 3.311218816873211e-05,
 '21165': 0.0,
 '1839': 0.00013280135026284037,
 '25910': 0.0008863478196508244,
 '17289': 0.0,
 '8768': 0.0033150502975890345,
 '9937': 5.155990390290855e-05,
 '16210': 0.0019454057179552982,
 '23382': 0.011593434448890438,
 '4068': 0.001705966808582797,
 '20478': 0.018718544821528512,
 '23038': 0.016810541147470326,
 '24582': 0.0,
 '14924': 0.007411260044394293,
 '8589': 0.000251832356

In [5]:
val_neg_list = []
for i in range(len(val_neg)):
    val_neg_list.append(val_neg[0,i])
    val_neg_list.append(val_neg[1,i])
for node in val_neg_list:
    if not G.has_node(node):
        print(node)

16039
24601
21450
12492


In [6]:
test_list = []
for i in range(len(test[0])):
    test_list.append(tuple(test[:,i]))

In [7]:
preds = nx.jaccard_coefficient(G, test_list) 

In [8]:
for i in test[0]:
    if not G.has_node(str(i)):
        print(i)
for i in test[1]:
    if not G.has_node(str(i)):
        print(i)

In [51]:
from collections import defaultdict
def getNeighbors(data):
    neighbors = defaultdict(set)
    for pair in data:
        neighbors[pair[0]].add(pair[1])
        neighbors[pair[1]].add(pair[0])
    return neighbors

def computeProximityScore(author1, author2, measure, neighbors):
    score = 0
    if measure == 'Jaccard':
        score = len(neighbors[author1].intersection(neighbors[author2]))
        return float(score)/len(neighbors[author1].union(neighbors[author2]))
    elif measure == 'Preferential':
        return len(neighbors[author1])*len(neighbors[author2])
    elif measure == 'adar':
        return sum([1.0/math.log(len(neighbors[v])) for v in neighbors[author1].intersection(neighbors[author2])])

def evaluation(toplist, groudtruth):
    count = 0
    for link in toplist:
        if link in groudtruth:
            count += 1
    print(float(count)/len(groudtruth))

In [52]:
df_train = pd.read_csv('training.txt', sep = '\s', header=None)
train = np.array([df_train[0], df_train[1]])
train_list = []
for i in range(len(train[0])):
    train_list.append(tuple(train[:,i]))
neighbors = getNeighbors(train_list)

C:\Users\simon\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [55]:
val_list = []
for i in range(len(val_neg[0])):
    val_list.append(tuple(val_neg[:,i]))
val_pos_list = []
for i in range(len(val_pos[0])):
    val_pos_list.append(tuple(val_pos[:,i]))
    val_list.append(tuple(val_pos[:,i]))

Jscore = {}
for i in range(len(val_list)):
    Jscore[val_list[i]] = computeProximityScore(val_list[i][0], val_list[i][1], 'adar', neighbors)
top_list = []
for i in range(100):
    max_score = max(Jscore, key=Jscore.get)
    top_list.append(max_score)
    Jscore.pop(max_score)
evaluation(top_list, val_pos_list)

0.73


In [13]:
import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import FullBatchLinkGenerator
from stellargraph.layer import GCN, LinkEmbedding


from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection

from stellargraph import globalvar
from stellargraph import datasets
from IPython.display import display, HTML
%matplotlib inline

import pandas as pd
import numpy as np

In [250]:
df_train = pd.read_csv('training.txt', sep = '\s', header=None)
train_edges = pd.DataFrame({"source":df_train[0], "target":df_train[1]})
graph = sg.StellarGraph(edges=train_edges)
generatorr = sg.mapper.FullBatchLinkGenerator(G_train, method='gcn')
#train_flow = train_gen.flow(list(df_train[0]), list(df_train[1]))

Using GCN (local pooling) filters...


C:\Users\simon\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [270]:
# two layers of GCN, each with hidden dimension 16
gcn = sg.layer.GCN(layer_sizes=[16, 16], 
                   activations=["relu", "relu"], 
                   generator=generator, dropout=0.3)
x_inp, x_out = gcn.in_out_tensors() # create the input and output TensorFlow tensors

In [271]:
x_out

<tf.Tensor 'gather_indices_5/Identity:0' shape=(1, None, 16) dtype=float32>

In [59]:
G.nodes()

NodeView(('8193', '16056', '24578', '21968', '18297', '16770', '17038', '8195', '2072', '20568', '15258', '19739', '16389', '21379', '11015', '23051', '20116', '15126', '21281', '17828', '1265', '21165', '1839', '25910', '17289', '8768', '9937', '16210', '23382', '4068', '20478', '23038', '24582', '14924', '8589', '24583', '8200', '23836', '12302', '16393', '17330', '15580', '24587', '7007', '24589', '5385', '9090', '17543', '8206', '15066', '8208', '13008', '25379', '24593', '18122', '8541', '24594', '2339', '8211', '20059', '4550', '4365', '1289', '6830', '25346', '21508', '20108', '21012', '15003', '17692', '25758', '22691', '15659', '3372', '8879', '12851', '20635', '2741', '9785', '570', '19961', '4164', '4513', '20562', '14540', '12365', '18894', '6179', '14807', '19423', '12496', '1653', '17655', '7956', '23293', '22', '106', '11183', '19440', '15793', '22618', '8215', '2912', '10211', '15300', '24', '23161', '3858', '19517', '15774', '7093', '24097', '22375', '26', '13096', '13

In [272]:
prediction = LinkEmbedding(activation="relu", method="ip")(x_out)

TypeError: in user code:

    C:\Users\simon\Anaconda3\lib\site-packages\stellargraph\layer\link_inference.py:154 call  *
        if int(x.shape[self.axis]) != 2:
    C:\Users\simon\Anaconda3\lib\site-packages\tensorflow\python\autograph\operators\py_builtins.py:207 int_  **
        return _py_int(x, base)
    C:\Users\simon\Anaconda3\lib\site-packages\tensorflow\python\autograph\operators\py_builtins.py:222 _py_int
        return int(x)

    TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'


In [208]:
import pandas as pd
import numpy as np
df_val_pos = pd.read_csv('val_positive.txt', sep = '\s', header=None)
df_val_neg = pd.read_csv('val_negative.txt', sep = '\s', header=None)
df_val = df_val_pos.append(df_val_neg)

C:\Users\simon\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\simon\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


In [240]:
edge_splitter_train = EdgeSplitter(G_train)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G_test, and obtain the
# reduced graph G_train with the sampled links removed:
G_train, edge_ids_train, edge_labels_train = edge_splitter_train.train_test_split(
    p=0.1, method="global", keep_connected=True
)
train_gen = FullBatchLinkGenerator(G_train, method="gcn")
train_flow = train_gen.flow(edge_ids_train, edge_labels_train)

IndexError: single positional indexer is out-of-bounds

In [212]:
edge_ids_val = np.array(df_val)
labels_val = []
for i in range(len(df_val_pos[0])):
    labels_val.append(1)
for i in range(len(df_val_neg[0])):
    labels_val.append(0)
edge_labels_val = np.array(labels_val)

array([1, 1, 1, ..., 0, 0, 0])

In [216]:
import networkx as nx
G_train = nx.read_edgelist('training.txt')
labes_train = []
for i in range(len(G_train.nodes())):
    labes_train.append(1)
edge_labels_train = np.array(labes_train)
df_train = pd.read_csv('training.txt', sep = '\s', header=None)
edge_ids_train = np.array(df_train)

C:\Users\simon\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys
